# Create centerline road network 
Based on BGT data

In [ ]:
# Standard library and path imports
import set_path
import warnings

# Third-party library imports
import shapely.ops as so
import geopandas as gpd
from geopandas import GeoDataFrame
from tqdm.notebook import tqdm
tqdm.pandas()
import folium

# Local or project-specific imports
import bgt_utils
import plot_utils
import poly_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [ ]:
if st.my_run == "azure":
    os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

## Import BGT data

In [ ]:
# Get BGT data
df_bgt_full = bgt_utils.get_bgt_data_for_bbox(st.bbox, st.bgt_road_layers)
df_bgt_full['naam'].value_counts()

## Select pilot areas

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

In [ ]:
# Only keep BGT data that intersects with pilot area
df_bgt = df_bgt_full.sjoin(df_areas, how='inner', predicate='intersects')
df_bgt['naam_left'].value_counts()

## Pre-process BGT data

In [ ]:
# Merge road polygons
df_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry'].unary_union))
df_bgt = gpd.GeoDataFrame(df_bgt.geometry.explode(index_parts=True))

## Create centerlines

In [ ]:
# Calculate centerlines
df_bgt['centerlines'] = df_bgt.progress_apply(
   lambda row: poly_utils.get_centerlines(row.geometry, interpolation_distance=2), axis=1)


df_bgt['geometry'] = df_bgt['centerlines']
df_bgt = df_bgt.drop(columns=['centerlines'])
df_bgt = df_bgt.loc[df_bgt['geometry'].notna()]

In [ ]:
# Merge lines
idxs = df_bgt.index[df_bgt.geom_type != 'LineString'].tolist()
df_bgt.loc[idxs, 'geometry'] = df_bgt.loc[idxs, 'geometry'].progress_apply(so.linemerge)

# Remove death-ends
df_bgt['geometry'] = df_bgt['geometry'].progress_apply(
    lambda x: poly_utils.remove_short_lines(x, st.min_se_length_fw))

In [ ]:
# Get each centerline in separate row
df_bgt_exp = df_bgt.explode(ignore_index=True)
df_bgt_exp.reset_index()
df_bgt_exp['road_part_id'] = df_bgt_exp.index
df_bgt_exp = df_bgt_exp.set_crs(crs=st.CRS)

## Store final output

In [ ]:
# Write road network to file
df_bgt_exp.to_file(cf.output_road_network, driver='GPKG')

### Visualize

In [ ]:
# set True for satelite background, False for regular background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add network and new edges
geo_j = folium.GeoJson(df_bgt_exp, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)

In [ ]:
# Add title
loc = 'Road network'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)

map.get_root().html.add_child(folium.Element(title_html))

In [ ]:
map